# **Weather Load Transform Clean and extract**

## Objectives

* Investigate world weather data, clean for US and targetted measurements, save out to zip file for use in Dashboard

## Inputs

* ghcnd yearly weather files and station_list

## Outputs

* File per year for US and target elements, pivotted to have 1 row per station per day
* unified file for US for 2000-2016 limited to stations within 10km of cities from the pollution data.

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
import pandas as pd
current_dir = os.getcwd()
print(current_dir)
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
print(current_dir)

h:\VScode\March Group\March_Team_Project\jupyter_notebooks
You set a new current directory
h:\VScode\March Group\March_Team_Project


# Section 1

Section 1 content

In [ ]:
# load csv files from Not_to_be_shared_to_repo folder
# apply the following column names to the file, Station_ID, Date, DataValue, MFlag, QFlag, SFlag, ObsTime
weather_df = pd.read_csv('Not_to_be_shared_to_repo/2000.csv.gz', header=None)
weather_df.info()

YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
ELEMENT = 4 character indicator of element type 
DATA VALUE = 5 character data value for ELEMENT 
M-FLAG = 1 character Measurement Flag 
Q-FLAG = 1 character Quality Flag 
S-FLAG = 1 character Source Flag 
OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)

In [ ]:
weather_df.columns = ["Station_ID", "Date", "Element","DataValue", "MFlag", "QFlag", "SFlag", "ObsTime"]
# set the column names
weather_df.head()

Load list of Stations from ghcnd=stations.txt, apply column headings and drop unrequired columns then view data

In [ ]:
# load the ghcnd-stations.txt file - fixed width file no header
stations_df = pd.read_fwf('Source_Data/ghcnd-stations.txt', header=None)
stations_df.columns = [ "StationId","Latitude", "Longitude", "Elevation", "Name", "GSN_Flag", "HCN_CRN_Flag", "WMO_ID"]
# Drop GSN_Flag, HCN_CRN_Flag, WMO_ID columns
stations_df.drop(["GSN_Flag", "HCN_CRN_Flag", "WMO_ID"], axis=1, inplace=True)
stations_df

Load countries data, apply column names, then load states data, apply column names

In [ ]:
# load ghcnd-countries.txt file - fixed width file no header
countries_df = pd.read_fwf('Source_Data/ghcnd-countries.txt', header=None)
countries_df.columns = ["CountryCode", "Name"]

# Load ghcnd-states.txt file - fixed width file no header
states_df = pd.read_fwf('Source_Data/ghcnd-states.txt', header=None)
states_df.columns = ["StateCode", "Name"]

print(countries_df.head())
print(states_df.head())

---

Remove country id from beginning of station_id - as explained in GHCND_readme.txt, then drop unrequired columns from data

In [ ]:
# add column to weather_df for the country code = set it to the first 2 characters of the Station_ID
weather_df['CountryCode'] = weather_df['Station_ID'].str[:2]
weather_df.head()
# drop weather_df MFlag, QFlag, SFlag, ObsTime columns
weather_df.drop(["MFlag", "QFlag", "SFlag", "ObsTime"], axis=1, inplace=True)

Calculate records by Country and deduplicate the Weather data

In [ ]:
# count of rows in weather_df by country
print(weather_df['CountryCode'].value_counts())
# deduplicate the weather_df by station_id, element, date, taking the first record if multiple records exist
weather_df = weather_df.drop_duplicates(subset=["Station_ID", "Element", "Date"], keep='first')
# count of rows in weather_df by country
print(weather_df['CountryCode'].value_counts())

As no duplications identified = this step can be ignored in the future.

From stationid extract the country code and count records

In [ ]:
# seperate the countryid out of the stations_df based on 1st 2 chracters of the StationId
stations_df['CountryCode'] = stations_df['StationId'].str[:2]
# count stations by country
print(stations_df['CountryCode'].value_counts())

Confirm that there are no rogue country codes in the stations_df, then filter both files for data from US only , then produce list of Elements in weather data for US records

In [ ]:
# check that country code is only country code
# count of rows where CountryCode in stations_df is not in countries_df
print(stations_df[~stations_df['CountryCode'].isin(countries_df['CountryCode'])].shape[0])
# count of rows in weather_df where CountryCode is not in countries_df
print(weather_df[~weather_df['CountryCode'].isin(countries_df['CountryCode'])].shape[0])
# count rows in stations_df
print(stations_df.shape[0])
# count rows in weather_df
print(weather_df.shape[0])
# filter stations_df to only include rows where CountryCode ="US"
stations_df = stations_df[stations_df['CountryCode']=="US"]
# filter weather_df to only include rows where CountryCode ="US"
weather_df = weather_df[weather_df['CountryCode']=="US"]
# count rows in stations_df
print(stations_df.shape[0])
# count rows in weather_df
print(weather_df.shape[0])
# count of Elements in weather_df
print(weather_df['Element'].value_counts())
# list of elements with at data in at least 90% of dates and stations
elements = weather_df['Element'].value_counts()
elements = elements[elements > 0.9 * 365 * 10]
elements = elements.index.tolist()
print(elements)


Filter weather for chosen elements, pivot data to produce 1 line per station per day and then perform a comparison of rows containing all elements and those only with the chosen elements

In [ ]:
ElementList = [
  # "PRCP",  # Precipitation (tenths of mm)
  # "WT03",  # Thunder
  "WT16",  # Rain (may include freezing rain, drizzle, and freezing drizzle)

  # "SNOW",  # Snowfall (mm)
  # "SNWD",  # Snow depth (mm)
  # "WESD",  # Water equivalent of snow on the ground (tenths of mm)
  # 'WESF',  # Water equivalent of snowfall (tenths of mm)
  "WT01",  # Fog, ice fog, or freezing fog (may include heavy fog)
  "WT04",  # Ice pellets, sleet, snow pellets, or small hail"
  "WT18",  # Snow, snow pellets, snow grains, or ice crystals
  'WT22',  # Ice fog or freezing fog
  "WT09",  # Blowing or drifting snow
  
  "TMAX",  # Maximum temperature (tenths of degrees C)
  "TMIN",  # Minimum temperature (tenths of degrees C)
  "TAVG",  # Average temperature (tenths of degrees C)

  "TSUN",  # Daily total sunshine (minutes)

  "AWND",  # Average daily wind speed (tenths of meters per second)
  # "WDF2",  # Direction of fastest 2-minute wind (degrees)
  # "WDF5",  # Direction of fastest 5-second wind (degrees)
  # "WSF2",  # Fastest 2-minute wind speed (tenths of meters per second)
  # "WSF5",  # Fastest 5-second wind speed (tenths of meters per second)
  # "FMTM",  # Time of fastest mile or fastest 1-minute wind (hours and minutes, i.e., HHMM)
  "PGTM",  # Peak gust time (hours and minutes, i.e., HHMM)
  # "WDMV",  # 24-hour wind movement (km)
  'WT11',  # High or damaging winds
  # "WT06",  # Glaze or rime
  # "WT08",  # Smoke or haze
  # "WT05",  # Hail (may include small hail)
  # "WT02",  # Heavy fog or heaving freezing fog (not always distinguished from fog)
  # "WT13",  # Mist
  ]

# save Stations_df to a csv file called Us_Stations.csv
stations_df.to_csv("Us_Stations.csv", index=False)
# filter weather_df to only include rows where Element in ElementList
weather_df = weather_df[weather_df['Element'].isin(ElementList)]
# count rows in weather_df
print(weather_df.shape[0])
# unpack weather_df Element in to seperate columns with value eq to DataValue where Element = TMAX, grouped by Station_Id and Date
weather_up_df = weather_df.pivot_table(index=["Station_ID", "Date"],
                     columns="Element",
                     values="DataValue",
                     aggfunc='first').reset_index()

# Ensure a column exists for each value in ElementList
for element in ElementList:
  if element not in weather_up_df.columns:
    weather_up_df[element] = None
# count rows of weather_df
print(weather_up_df.shape[0])
# save weather_df to a csv file called Us_Weather_Unpacked.zip
weather_up_df.to_csv("Us_Weather_Unpacked.zip", index=False, compression='zip')

Quick look at a single station to confirm that the pivot_table is working correctly

In [ ]:
weather_up_df[weather_up_df["Station_ID"] == "USW00094173"].head(100)

Do the same for a single element in the original data format

In [ ]:
# weather_df.head(100)
weather_df[weather_df["Element"] == "WT01"].head(100)

# Scripts to bulk load and clean yearly data files using techniks from above

Load annual data, filter for US and required elements, drop unrequired columns, perform pivot and resave as a zip file, print file name created and row/column counts

In [ ]:
ElementList = [
  "WT16",  # Rain (may include freezing rain, drizzle, and freezing drizzle)
  "WT01",  # Fog, ice fog, or freezing fog (may include heavy fog)
  "WT04",  # Ice pellets, sleet, snow pellets, or small hail"
  "WT18",  # Snow, snow pellets, snow grains, or ice crystals
  'WT22',  # Ice fog or freezing fog
  "WT09",  # Blowing or drifting snow
  "TMAX",  # Maximum temperature (tenths of degrees C)
  "TMIN",  # Minimum temperature (tenths of degrees C)
  "TAVG",  # Average temperature (tenths of degrees C)
  "TSUN",  # Daily total sunshine (minutes)
  "AWND",  # Average daily wind speed (tenths of meters per second)
  "PGTM",  # Peak gust time (hours and minutes, i.e., HHMM)
  'WT11',  # High or damaging winds
  ]

weather_df  = pd.DataFrame()
weather_up_df = pd.DataFrame()

# list of files to load
FileToLoad = ["2000.csv.gz",
              "2001.csv.gz",
              "2002.csv.gz",
              "2003.csv.gz",
              "2004.csv.gz",
              "2005.csv.gz",
              "2006.csv.gz",
              "2007.csv.gz",
              "2008.csv.gz",
              "2009.csv.gz",
              "2010.csv.gz",
              "2011.csv.gz",
              "2012.csv.gz",
              "2013.csv.gz",
              "2014.csv.gz",
              "2015.csv.gz",
              "2016.csv.gz",
              ]

# load a file from the list above then process it and append it to weather_df
for file in FileToLoad:
    # add filename to path "Not_to_be_shared_to_repo/"
    filepath = os.path.join("Not_to_be_shared_to_repo", file)
    # setup df
    weather_df  = pd.DataFrame()
    weather_up_df = pd.DataFrame()
    # Print filepath
    print(filepath)
    # load the file
    weather_df = pd.read_csv(filepath, header=None)
    # Name columns
    weather_df.columns = ["Station_ID", "Date", "Element", "DataValue", "MFlag", "QFlag", "SFlag", "ObsTime"]
    # Extract Country Code
    weather_df['CountryCode'] = weather_df['Station_ID'].str[:2]
    # Drop unneccesary columns
    weather_df.drop(["MFlag", "QFlag", "SFlag", "ObsTime"], axis=1, inplace=True)
    # Deduplicate weather_df
    weather_df = weather_df.drop_duplicates(subset=["Station_ID", "Element", "Date"], keep='first')
    # filter weather_df to only include rows where CountryCode ="US"
    weather_df = weather_df[weather_df['CountryCode']=="US"]
    # filter weather_df to only include rows where Element in ElementList
    weather_df = weather_df[weather_df['Element'].isin(ElementList)]
    # unpack weather_df Element in to seperate columns with value eq to DataValue where Element = TMAX, grouped by Station and Date
    weather_df = weather_df.pivot_table(index=["Station_ID", "Date"],
                                        columns="Element",
                                        values="DataValue",
                                        aggfunc='first').reset_index()
    # print the shape of weather_df
    print(weather_df.shape)
    # add filename to path "Not_to_be_shared_to_repo/"
    filename = f"Us_{file}_Weather_Unpacked.zip"
    filepath = os.path.join("Not_to_be_shared_to_repo", filename)
    # implement save append to csv file
    weather_df.to_csv(filepath, index=True, mode='a', header=True, compression='zip')
    # drop weather_df and weather_up_df
    del weather_df
    # print file name

Load pivotted, filtered data by year and filter for required cities - append into single file and save.

In [2]:

FileToLoad = ["Us_2000.csv.gz_Weather_Unpacked.zip",
              "Us_2001.csv.gz_Weather_Unpacked.zip",
              "Us_2002.csv.gz_Weather_Unpacked.zip",
              "Us_2003.csv.gz_Weather_Unpacked.zip",
              "Us_2004.csv.gz_Weather_Unpacked.zip",
              "Us_2005.csv.gz_Weather_Unpacked.zip",
              "Us_2006.csv.gz_Weather_Unpacked.zip",
              "Us_2007.csv.gz_Weather_Unpacked.zip",
              "Us_2008.csv.gz_Weather_Unpacked.zip",
              "Us_2009.csv.gz_Weather_Unpacked.zip",
              "Us_2010.csv.gz_Weather_Unpacked.zip",
              "Us_2011.csv.gz_Weather_Unpacked.zip",
              "Us_2012.csv.gz_Weather_Unpacked.zip",
              "Us_2013.csv.gz_Weather_Unpacked.zip",
              "Us_2014.csv.gz_Weather_Unpacked.zip",
              "Us_2015.csv.gz_Weather_Unpacked.zip",
              "Us_2016.csv.gz_Weather_Unpacked.zip",
              ]

weather_df = pd.DataFrame()
weather_up_df = pd.DataFrame()
# load weather stations within 10Km of pollution data cities
stations_df = pd.read_csv("Source_Data\\Us_Stations_with_City_10km.csv")


# load file from list and append to weather_final_df
for file in FileToLoad:
    # set path for fileload
    loadfile = "Not_to_be_shared_to_repo/"
    filepath = os.path.join(loadfile, file)
    # load file
    weather_df = pd.read_csv(filepath)
    # filter to only keep stations that are in stations_df
    weather_df = weather_df[weather_df['Station_ID'].isin(stations_df['StationId'])]
    # append to weather_final_df
    weather_up_df = pd.concat([weather_up_df, weather_df], ignore_index=True)
    # print file name
    print(file)
    # print shape of weather_final_df
    print(weather_up_df.shape)
    # drop weather_df

# save weather_final_df to a csv file called Us_Weather_Final.csv
weather_up_df.to_csv("Not_to_be_shared_to_repo/Us_Weather_Final_10km.zip", index=True, header=True, compression='zip')

Us_2000.csv.gz_Weather_Unpacked.zip
(14842, 16)
Us_2001.csv.gz_Weather_Unpacked.zip
(29554, 16)
Us_2002.csv.gz_Weather_Unpacked.zip
(43070, 16)
Us_2003.csv.gz_Weather_Unpacked.zip
(57019, 16)
Us_2004.csv.gz_Weather_Unpacked.zip
(71274, 16)
Us_2005.csv.gz_Weather_Unpacked.zip
(85939, 16)
Us_2006.csv.gz_Weather_Unpacked.zip
(100777, 16)
Us_2007.csv.gz_Weather_Unpacked.zip
(115989, 16)
Us_2008.csv.gz_Weather_Unpacked.zip
(131638, 16)
Us_2009.csv.gz_Weather_Unpacked.zip
(147930, 16)
Us_2010.csv.gz_Weather_Unpacked.zip
(164269, 16)
Us_2011.csv.gz_Weather_Unpacked.zip
(181361, 16)
Us_2012.csv.gz_Weather_Unpacked.zip
(198545, 16)
Us_2013.csv.gz_Weather_Unpacked.zip
(215154, 16)
Us_2014.csv.gz_Weather_Unpacked.zip
(231841, 16)
Us_2015.csv.gz_Weather_Unpacked.zip
(248608, 16)
Us_2016.csv.gz_Weather_Unpacked.zip
(265147, 16)


---

In [ ]:
import pandas as pd

# import csv from source/simplemaps-worldcities-basic.csv
cities_df = pd.read_csv("Source_Data\\simplemaps_uscities_basicv1.90.zip")
# import csv pollution_data_available.csv
pollution_df = pd.read_csv("pollution_data_available.csv")
# update pollution_df with lat and long from cities_df
pollution_df = pollution_df.merge(cities_df, left_on='City', right_on='city', how='left')
#show data
pollution_df
# save pollution_df to a csv file called city_data_inc_latlog_.csv
pollution_df.to_csv("city_data_inc_latlog_.csv", index=False)


Augment the Pollution Dataset with Coordinates:

Retrieve the latitude and longitude for each city in the pollution dataset. Resources like the SimpleMaps US Cities Database provide comprehensive data on U.S. cities, including their geographic coordinates.

In [ ]:
# Load the city_data_inc_latlog_.csv file
city_df = pd.read_csv("city_data_inc_latlog_.csv")
#drop unneccesary columns
# city_df.drop(["city", "admin_name", "population", "id", "zips", "County", "State", "incorporated", "source", "ranking", "timezone"], axis=1, inplace=True)
#show data
city_df

Calculate Distances Between Cities and Stations:

Utilize the Haversine formula to compute the great-circle distance between each city and all monitoring stations. This formula calculates the shortest distance over the Earth's surface, providing an accurate measure between two points specified by their latitude and longitude.

In [ ]:
import numpy as np
# load US_Stations.csv
stations_df = pd.read_csv("Source_Data\\Us_Stations.csv")
#

# Function to calculate Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Distance in km

# using the lat and long from the stations_df and city_df calculate the distance between the two
stations_df['City'] = None
stations_df['CityDistance'] = None
for i, station in stations_df.iterrows():
    lat1, lon1 = station['Latitude'], station['Longitude']
    min_distance = np.inf
    for j, city in city_df.iterrows():
        lat2, lon2 = city['lat'], city['lng']
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance < min_distance:
            min_distance = distance
            stations_df.at[i, 'City'] = city['city']
            stations_df.at[i, 'CityDistance'] = min_distance

# display data
stations_df

In [ ]:
# save stations_df to a csv file called Us_Stations_with_City.csv
stations_df.to_csv("Source_Data\\Us_Stations_with_City.csv", index=False)


In [ ]:
# filter stations_df to provide only the stations that are within 10km of a city
stations_df = stations_df[stations_df['CityDistance'] < 10]
# display data
stations_df

In [ ]:
# save stations_df to a csv file called Us_Stations_with_City_10km.csv
stations_df.to_csv("Source_Data\\Us_Stations_with_City_10km.csv", index=False)